<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Proteins-with-true-single-internal-cysteine" data-toc-modified-id="Proteins-with-true-single-internal-cysteine-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Proteins with true single internal cysteine</a></span></li><li><span><a href="#Remove-cysteines-in-disulfide-bonds" data-toc-modified-id="Remove-cysteines-in-disulfide-bonds-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Remove cysteines in disulfide bonds</a></span></li><li><span><a href="#Annotate-CCC/CXC/CC/C-motifs-and-peptides" data-toc-modified-id="Annotate-CCC/CXC/CC/C-motifs-and-peptides-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Annotate CCC/CXC/CC/C motifs and peptides</a></span></li><li><span><a href="#Hypergeometric-distribution-and-Fisher's-exact-test-for-internal-C/CC/CXC/CCC-motifs" data-toc-modified-id="Hypergeometric-distribution-and-Fisher's-exact-test-for-internal-C/CC/CXC/CCC-motifs-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Hypergeometric distribution and Fisher's exact test for internal C/CC/CXC/CCC motifs</a></span></li></ul></div>

In [ ]:
import sys
import os
import session_info

# Add the '0_functions' folder to sys.path
sys.path.append(os.path.join(os.getcwd(), '..', '0_functions'))

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import fisher_exact
from functions import Urn
from functions import add_Cpos
from functions import get_Ccount
from functions import pep_intern
from functions import annotate_pep_internal

In [2]:
# Display session information
session_info.show()

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
datafolder = 'data'
figures = 'data/figures'
PSSMSearch = 'data/PssmSearch'

In [5]:
# Load data

# clean FASTA file of all identified proteins
fasta = pd.read_csv(os.path.join(datafolder, 'identified_proteins', 'final_fasta_seqs_identified_prots.csv'), sep=',')

# Cysteines in disulfide bonds
bonded = pd.read_csv(os.path.join(datafolder, 'UniProt_SPARQL_queries', 'up_output_multi_disulfid_16.01.23.csv'), sep=';')

# Proteins with true single internal cysteine 

In [6]:
int_F = fasta[fasta['motif'] == 'internal'].reset_index(drop=True)

In [7]:
print('Internal farnesylated only:', len(int_F))

Internal farnesylated only: 6


In [8]:
int_F

,ID,name,seqID,seq,len,Ccount,Cpos,motif,pep
0,Q9UI42,CBPA4,sp|Q9UI42|CBPA4_HUMAN,MRWILFIGALIGSSICGQEKFFGDQVLRINVRNGDEISKLSQLVNS...,421,4,NaN,internal,NaN
1,P41567,EIF1,sp|P41567|EIF1_HUMAN,MSAIQNLHSFDPFADASKGDDLLPAGTEDYIHIRIQQRNGRKTLTT...,113,2,NaN,internal,NaN
2,P52292,IMA1,sp|P52292|IMA1_HUMAN,MSTNENANTPAARLHRFKNKGKDSTEMRRRRIEVNVELRKAKKDDQ...,529,6,NaN,internal,NaN
3,Q8IUC0,KR131,sp|Q8IUC0|KR131_HUMAN,MSYNCCSGNFSSRSCGGYLHYPASSCGFSYPSNQVYSTDLCSPSTC...,172,21,NaN,internal,NaN
4,Q8N0X7,SPART,sp|Q8N0X7|SPART_HUMAN,MEQEPQNGEPAEIKIIREAYKKAFLFVNKGLNTDELGQKEEAKNYY...,666,10,NaN,internal,NaN
5,Q969T9,WBP2,sp|Q969T9|WBP2_HUMAN,MALNKNHSEGGGVIVNNTESILMSYDHVELTFNDMKNVPEAFKGTK...,261,2,NaN,internal,NaN


In [9]:
# check for proteins with only 1 possibly prenylated C that is internal
F_int_True_C1 = int_F[int_F['Ccount'] == 1]

# Get Uniprot annotations for proteins with 1 cysteine
#F_int_True_C1_UP = F_found[F_found['ID'].isin(F_int_True_C1['ID'])].reset_index(drop=True)

print('N of F proteins with exactly 1 cysteine:', len(F_int_True_C1))

N of F proteins with exactly 1 cysteine: 0


# Remove cysteines in disulfide bonds

In [11]:
# remove Cpos column
dataframes = [int_F] 

for df in dataframes:
    if 'Cpos' in df.columns:
        df.drop('Cpos', axis=1, inplace=True)

In [12]:
# Add C positions counting from N-terminal and counting from C-terminal
# ATTENTION: these positions start from 1, not 0, as would be necessary for calling the Cs on index
fasta_F = add_Cpos(int_F)

In [13]:
# add C count 
bonded = bonded.rename(columns={'bond_Cpos': 'N_Cpos'}).drop_duplicates()
agg_bonded_Ccount = get_Ccount(bonded)

cols = ['ID', 'seqID', 'seq', 'len', 'Cpos', 'Ccount', 'motif', 'N_Cpos']

discardF = bonded.merge(fasta_F, on=['ID', 'N_Cpos'])
discardF = discardF[cols]

print('Before removing cysteines in disulfide bonds:')
print('Number of Cs in F proteins:', len(fasta_F)) # check

fasta_F = fasta_F[cols]

fasta_F = pd.concat([fasta_F, discardF]).drop_duplicates(keep=False).reset_index(drop=True)

print('\nAfter removing cysteines in disulfide bonds:')
print('Number of Cs in F proteins:', len(fasta_F)) # check

Before removing cysteines in disulfide bonds:
Number of Cs in F proteins: 45

After removing cysteines in disulfide bonds:
Number of Cs in F proteins: 43


In [14]:
print('Number of F proteins', len(fasta_F.groupby('ID').agg(set).reset_index())) # check

Number of F proteins 6


In [15]:
# check if and how much the possibly prenylated Cs have been reduced to 1 possibly prenylated C with this method

# F
agg_F_bonded_filt_Ccount = get_Ccount(fasta_F)
F_bonded_C1 = agg_F_bonded_filt_Ccount[agg_F_bonded_filt_Ccount['Ccount'] == 1]
F_bonded_C1 = F_bonded_C1[~F_bonded_C1['ID'].isin(F_int_True_C1['ID'])].reset_index(drop=True)
F_bonded_C1

,Ccount,ID,seqID,seq,len,Cpos,motif,N_Cpos,Count_all


In [16]:
# Actualize Ccount after removing cysteines in disulfide bonds

# Replace 'Ccount' values in 'fasta_F_GG' with values from 'agg_F_GG_bonded_filt_Ccount' based on matching 'ID'
fasta_F['Ccount'] = fasta_F['ID'].map(agg_F_bonded_filt_Ccount.set_index('ID')['Ccount'])

In [17]:
print('N of F proteins with exactly 1 cysteine, after removing cysteines in disulfide bonds:', 
      len(fasta_F['ID'][fasta_F['Ccount'] == 1].unique()))

N of F proteins with exactly 1 cysteine, after removing cysteines in disulfide bonds: 0


In [18]:
# Add count of all cysteines (Count_all), including those in disulfide bonds, to fasta dfs from aggregated dfs

fasta_F = fasta_F.merge(agg_F_bonded_filt_Ccount[['ID', 'Count_all']], on='ID', how='left')

In [19]:
fasta_F

,ID,seqID,seq,len,Cpos,Ccount,motif,N_Cpos,Count_all
0,Q9UI42,sp|Q9UI42|CBPA4_HUMAN,MRWILFIGALIGSSICGQEKFFGDQVLRINVRNGDEISKLSQLVNS...,421,-406,2,internal,16,4
1,Q9UI42,sp|Q9UI42|CBPA4_HUMAN,MRWILFIGALIGSSICGQEKFFGDQVLRINVRNGDEISKLSQLVNS...,421,-66,2,internal,356,4
2,P41567,sp|P41567|EIF1_HUMAN,MSAIQNLHSFDPFADASKGDDLLPAGTEDYIHIRIQQRNGRKTLTT...,113,-45,2,internal,69,2
3,P41567,sp|P41567|EIF1_HUMAN,MSAIQNLHSFDPFADASKGDDLLPAGTEDYIHIRIQQRNGRKTLTT...,113,-20,2,internal,94,2
4,P52292,sp|P52292|IMA1_HUMAN,MSTNENANTPAARLHRFKNKGKDSTEMRRRRIEVNVELRKAKKDDQ...,529,-397,6,internal,133,6
5,P52292,sp|P52292|IMA1_HUMAN,MSTNENANTPAARLHRFKNKGKDSTEMRRRRIEVNVELRKAKKDDQ...,529,-307,6,internal,223,6
6,P52292,sp|P52292|IMA1_HUMAN,MSTNENANTPAARLHRFKNKGKDSTEMRRRRIEVNVELRKAKKDDQ...,529,-293,6,internal,237,6
7,P52292,sp|P52292|IMA1_HUMAN,MSTNENANTPAARLHRFKNKGKDSTEMRRRRIEVNVELRKAKKDDQ...,529,-258,6,internal,272,6
8,P52292,sp|P52292|IMA1_HUMAN,MSTNENANTPAARLHRFKNKGKDSTEMRRRRIEVNVELRKAKKDDQ...,529,-111,6,internal,419,6
9,P52292,sp|P52292|IMA1_HUMAN,MSTNENANTPAARLHRFKNKGKDSTEMRRRRIEVNVELRKAKKDDQ...,529,-63,6,internal,467,6


# Annotate CCC/CXC/CC/C motifs and peptides

In [20]:
# Check how many N-terminal cysteines have to be excluded from PSSMSearch and Foldseek display, due to too short peptides

print("F - N-terminal cysteines that can't be aligned for sequence logo plots:", len(fasta_F[fasta_F['N_Cpos'] < 6].reset_index(drop=True)))
fasta_F[fasta_F['N_Cpos'] < 6].reset_index(drop=True)

F - N-terminal cysteines that can't be aligned for sequence logo plots: 1


,ID,seqID,seq,len,Cpos,Ccount,motif,N_Cpos,Count_all
0,Q8IUC0,sp|Q8IUC0|KR131_HUMAN,MSYNCCSGNFSSRSCGGYLHYPASSCGFSYPSNQVYSTDLCSPSTC...,172,-168,21,internal,5,21


In [21]:
# add peptides for all internal C positions
F_motifs = pep_intern(fasta_F)

In [22]:
# annotate peptides according to motif CCC/CXC/CC/C
F_motifs = annotate_pep_internal(F_motifs)

In [23]:
# aggregate on different C motifs

F_motifs_C = F_motifs[~F_motifs['pepC'].isna()].reset_index(drop=True)
F_motifs_C1 = F_motifs[F_motifs['Ccount'] == 1].reset_index(drop=True)
F_motifs_Cplus = F_motifs[F_motifs['Ccount'] != 1].reset_index(drop=True)
F_motifs_Cplus = F_motifs_Cplus[~F_motifs_Cplus['pepC'].isna()].reset_index(drop=True)
F_motifs_CC = F_motifs[~F_motifs['pepCC'].isna()].reset_index(drop=True)
F_motifs_CXC = F_motifs[~F_motifs['pepCXC'].isna()].reset_index(drop=True)
F_motifs_CCC = F_motifs[~F_motifs['pepCCC'].isna()].reset_index(drop=True)

In [24]:
F_motifs_C1

,ID,seqID,seq,pep,len,Cpos,Ccount,motif,N_Cpos,Count_all,pepCCC,pepCC,pepCXC,pepC


In [25]:
print('Farnesylated: number of peptides with internal C')
print('C all: ', len(F_motifs_C), ', C1: ',  len(F_motifs_C1), ', Cplus: ',  len(F_motifs_Cplus), ', CC: ',  
      len(F_motifs_CC), ', CXC: ',  len(F_motifs_CXC),  ', CCC: ', len(F_motifs_CCC), sep='')

Farnesylated: number of peptides with internal C
C all: 40, C1: 0, Cplus: 40, CC: 0, CXC: 0, CCC: 0


In [26]:
print('Farnesylated: number of proteins with internal C')
print('C all: ', len(F_motifs_C['ID'].unique()), ', C1: ',  len(F_motifs_C1['ID'].unique()), ', Cplus: ',  len(F_motifs_Cplus['ID'].unique()), ', CC: ',  
      len(F_motifs_CC['ID'].unique()), ', CXC: ',  len(F_motifs_CXC['ID'].unique()),  ', CCC: ', len(F_motifs_CCC['ID'].unique()), sep='')

Farnesylated: number of proteins with internal C
C all: 6, C1: 0, Cplus: 6, CC: 0, CXC: 0, CCC: 0


In [27]:
# save peptides in txt format for pssmsearch
F_motifs_C.pepC.to_csv(os.path.join(PSSMSearch, 'internal', 'F_motifs_C.txt'), header=None, index=None, sep=' ')
F_motifs_C1.pepC.to_csv(os.path.join(PSSMSearch, 'internal', 'F_motifs_C1.txt'), header=None, index=None, sep=' ')
F_motifs_Cplus.pepC.to_csv(os.path.join(PSSMSearch, 'internal', 'F_motifs_Cplus.txt'), header=None, index=None, sep=' ')
F_motifs_CC.pepCC.to_csv(os.path.join(PSSMSearch, 'internal', 'F_motifs_CC.txt'), header=None, index=None, sep=' ')
F_motifs_CXC.pepCXC.to_csv(os.path.join(PSSMSearch, 'internal', 'F_motifs_CXC.txt'), header=None, index=None, sep=' ')
F_motifs_CCC.pepCCC.to_csv(os.path.join(PSSMSearch, 'internal', 'F_motifs_CCC.txt'), header=None, index=None, sep=' ')

In [28]:
# save all identified prenylated proteins with exactly one internal cysteine

#F_motifs_C1.to_csv(os.path.join(datafolder, 'identified_proteins', 'intern_Ccount_1.csv'), sep=',', index=False)

In [29]:
# save dfs for AlphaFold in RQ6

F_motifs.to_csv(os.path.join(datafolder, 'identified_proteins', 'F_internal.csv'), sep=',', index=False)

# Expected distribution and Fisher's exact test for internal C/CC/CXC/CCC motifs

In [30]:
# Attention! Keep in mind that we don't take into account 57 
# N-terminal peptides for the prenylated proteins and 66 for the background proteins!

In [31]:
F_int_motifs = np.array([len(F_motifs_C['ID']), len(F_motifs_CC['ID']), len(F_motifs_CXC['ID']), 
                len(F_motifs_CCC['ID'])])

In [32]:
back_int_motifs = np.array([5660, 139, 108, 5])

In [33]:
# Hypergeometric distribution

# Background: number of peptides with internal C
# C:5660, CC:139, CXC:108, CCC:5

# Total N of peptides with C: 5912

K_arr = [5660, 139, 108, 5]
urn = Urn(K_arr)

In [34]:
# Category labels
category_labels = ['C', 'CC', 'CXC', 'CCC']  # Category names

In [35]:
# List of DataFrames and their names
dataframes = [
    (F_int_motifs, 'F_int_motifs')
]

In [36]:
# Loop through each DataFrame and perform analysis
for df, df_name in dataframes:
    # Calculate total draws and expected counts
    n = sum(df)
    m, _ = urn.moments(n)

    # Prepare DataFrame results
    results = {
        'Category': category_labels,
        'Expected Distribution': [round(value) for value in m],
        'Actual Distribution': df,
        'P-Value': []
    }

    # Calculate p-values and add significance markers
    for i in range(len(df)):
        contingency_table = np.array([[df[i], m[i]],
                                       [sum(df) - df[i], sum(m) - m[i]]])
        _, p_value = fisher_exact(contingency_table)

        # Add significance markers
        if p_value < 0.001:
            p_value_str = "0.0****"
        elif p_value < 0.01:
            p_value_str = "0.0****"
        elif p_value < 0.05:
            p_value_str = f"{p_value:.2f}*"
        else:
            p_value_str = f"{p_value:.2f}"

        results['P-Value'].append(p_value_str)

    # Create DataFrame from results
    results_df = pd.DataFrame(results)

    # Transpose the DataFrame
    results_df = results_df.set_index('Category').T  # Set 'Category' as index and transpose

    # Display the results as a formatted table
    print(f"\nResults for {df_name}:")
    print(results_df)


Results for F_int_motifs:
Category                  C    CC   CXC   CCC
Expected Distribution    38     1     1     0
Actual Distribution      40     0     0     0
P-Value                0.49  1.00  1.00  1.00
